> Call wandb.init() at the top of your script to start a new run

Call wandb.init() once at the beginning of your script to initialize a new job. This creates a new run in W&B and launches a background process to sync data.

- On-Prem: If you need a private cloud or local instance of W&B, see our Self Hosted offerings.
- Automated Environments: Most of these settings can also be controlled via Environment Variables. This is often useful when you're running jobs on a cluster.

## Reference Documentation

View the reference docs for this function, generated from the wandb Python library.

In [ ]:
#|hide
import wandb

## Common Questions

### How do I launch multiple runs from one script?

If you're trying to start multiple runs from one script, add two things to your code:

1. run = wandb.init(reinit=True): Use this setting to allow reinitializing runs
2. run.finish(): Use this at the end of your run to finish logging for that run

In [ ]:
#|output: false
#|eval: false
import wandb
for x in range(2):
    run = wandb.init(reinit=True)
    for y in range (25):
        wandb.log({"metric": x+y})
    run.finish()

wandb: Currently logged in as: hamelsmu (github-wandb). Use `wandb login --relogin` to force relogin


metric,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
metric,24


metric,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
metric,25


Alternatively you can use a python context manager which will automatically finish logging:

In [ ]:
#|output: false
#|eval: false
import wandb
for x in range(2):
    run = wandb.init(reinit=True)
    with run:
        for y in range(25):
            run.log({"metric": x+y})

metric,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
metric,24


metric,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
metric,25


# InitStartError: Error communicating with wandb process

This error indicates that the library is having difficulty launching the process which synchronizes data to the server.
The following workarounds can help resolve the issue in certain environments:

::: {.panel-tabset}

### Linux / OS X

In [ ]:
#|eval: false
#|output: false

wandb.init(settings=wandb.Settings(start_method="fork"))

### Google Colab

For versions prior to 0.13.0 we suggest using:

In [ ]:
#|eval: false
#|output: false
wandb.init(settings=wandb.Settings(start_method="thread"))

:::

## How do I programmatically access the human-readable run name?

It's available as the .name attribute of a [wandb.Run](https://docs.wandb.ai/ref/python/run).

In [ ]:
#|output: false
import wandb
wandb.init();
run_name = wandb.run.name

In [ ]:
print(run_name)

sweet-sun-46


## Can I just set the run name to the run ID?

If you'd like to overwrite the run name (like snowy-owl-10) with the run ID (like qvlp96vk) you can use this snippet:

In [ ]:
#| output: false
import wandb
wandb.init();
wandb.run.name = wandb.run.id
wandb.run.save()

True

In [ ]:
print(wandb.run.name)

16ej8478


## Is it possible to save metrics offline and sync them to W&B later?

By default, wandb.init starts a process that syncs metrics in real time to our cloud hosted app. If your machine is offline, you don't have internet access, or you just want to hold off on the upload, here's how to run wandb in offline mode and sync later.
You'll need to set two environment variables.

1. `WANDB_API_KEY=$KEY`, where $KEY is the API Key from your settings page
2. `WANDB_MODE="offline"`

And here's a sample of what this would look like in your script:

In [ ]:
#|echo: false
%pycat offline.py

import wandb
import os

os.environ["WANDB_MODE"] = "offline"

config = {
  "dataset": "CIFAR10",
  "machine": "offline cluster",
  "model": "CNN",
  "learning_rate": 0.01,
  "batch_size": 128,
}

wandb.init(project="offline-demo")

for i in range(100):
  wandb.log({"accuracy": i})


In [ ]:
#|hide
!rm -rf wandb

Here's a sample terminal output:


In [ ]:
%%bash

python offline.py

wandb: Tracking run with wandb version 0.13.1
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb: accuracy ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: 
wandb: Run summary:
wandb: accuracy 99
wandb: 
wandb: You can sync this run to the cloud by running:
wandb: wandb sync /Users/hamel/wandb-nbdev/docs/wandb/offline-run-20220819_111619-1kj5i84c
wandb: Find logs at: ./wandb/offline-run-20220819_111619-1kj5i84c/logs


And once you're ready, just run a sync command to send that folder to the cloud with the following command:

`wandb sync wandb/dryrun-folder-name`

In [ ]:
#|eval: false
!wandb sync wandb/latest-run

Find logs at: /Users/hamel/wandb-nbdev/docs/wandb/debug-cli.hamel.log
Syncing: https://wandb.ai/github-wandb/offline-demo/runs/1kj5i84c ... done.
